<img style="float: right" src="img/saturn.png" width="300" />

# Scaling Machine Learning in Python

## Large datasets

- Load and process large dataset
    - `dask.dataframe`
- Predict over large dataset
    - `ParallelPostFit`
    - `map_partitions`
- Train model with large dataset
    - `Incremental`
    - `dask_ml`
    - XGBoost

# Load and process large dataset

## Initialize Dask cluster

In [1]:
from dask_saturn import SaturnCluster
from dask.distributed import Client

cluster = SaturnCluster()
client = Client(cluster)
client.wait_for_workers(5)

[2020-11-09 17:16:05] INFO - dask-saturn | Cluster is ready


## Load data

In [2]:
import s3fs
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

s3 = s3fs.S3FileSystem(anon=True)

In [3]:
import dask
import dask.dataframe as dd
from dask.distributed import wait

In [4]:
files_2019 = 's3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv'
s3.glob(files_2019)

['nyc-tlc/trip data/yellow_tripdata_2019-01.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-02.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-03.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-04.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-05.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-06.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-07.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-08.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-09.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-10.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-11.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-12.csv']

In [5]:
%%time

taxi = dd.read_csv(
    files_2019,
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
    assume_missing=True,
)

CPU times: user 38.8 ms, sys: 12.4 ms, total: 51.2 ms
Wall time: 150 ms


In [6]:
taxi

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=127,,,,,,,,,,,,,,,,,,
,float64,datetime64[ns],datetime64[ns],float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
taxi_bytes = taxi.memory_usage(deep=True).sum()
taxi_bytes

dd.Scalar<series-..., dtype=int64>

In [8]:
%%time
print(f"Size (MB): {taxi_bytes.compute() / 1e6}")

Size (MB): 16367.014316
CPU times: user 73.7 ms, sys: 753 µs, total: 74.5 ms
Wall time: 30.6 s


In [9]:
taxi = taxi.persist()

In [10]:
%%time
_ = wait(taxi)

CPU times: user 69.2 ms, sys: 4.46 ms, total: 73.7 ms
Wall time: 30 s


In [11]:
%%time
taxi_bytes = taxi.memory_usage(deep=True).sum()
print(f"Size (MB): {taxi_bytes.compute() / 1e6}")

Size (MB): 16367.014316
CPU times: user 46.5 ms, sys: 7.93 ms, total: 54.5 ms
Wall time: 1.58 s


# Exploratory analysis

In [25]:
%%time
taxi_describe = taxi.describe().compute().T
np.round(taxi_describe, 3)

CPU times: user 3.69 s, sys: 46.9 ms, total: 3.74 s
Wall time: 14.3 s


,count,mean,std,min,25%,50%,75%,max
VendorID,84152418.0,1.645,0.498,1.00,1.00,2.00,2.00,4.00
passenger_count,84152418.0,1.563,1.208,0.00,1.00,1.00,2.00,9.00
trip_distance,84399019.0,3.001,8.091,-37264.53,1.07,1.93,8.82,45977.22
RatecodeID,84152418.0,1.061,0.760,1.00,1.00,1.00,1.00,99.00
PULocationID,84399019.0,163.158,66.016,1.00,132.00,162.00,234.00,265.00
DOLocationID,84399019.0,161.353,70.251,1.00,116.00,163.00,236.00,265.00
payment_type,84152418.0,1.289,0.479,1.00,1.00,1.00,2.00,5.00
fare_amount,84399019.0,13.344,174.375,-1856.00,7.00,11.00,32.04,943274.80
extra,84399019.0,1.087,1.249,-60.00,0.00,1.00,3.00,535.38
mta_tax,84399019.0,0.495,0.067,-0.50,0.50,0.50,0.50,212.42


## Feature engineering

In [12]:
# specify feature and label column names
raw_features = [
    'tpep_pickup_datetime', 
    'passenger_count', 
    'tip_amount', 
    'fare_amount',
]
features = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
label = 'tip_fraction'

In [13]:
def prep_df(taxi_df):
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = taxi_df[taxi_df.fare_amount > 0][raw_features].copy()  # avoid divide-by-zero
    df[label] = df.tip_amount / df.fare_amount
     
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [label]].astype(float).fillna(-1)
    
    return df

In [14]:
taxi_feat = prep_df(taxi)
taxi_feat.head()

,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,tip_fraction
0,1.0,1.0,0.0,24.0,46.0,1.0,0.235714
1,1.0,1.0,0.0,24.0,59.0,1.0,0.071429
2,4.0,51.0,13.0,109.0,48.0,3.0,0.000000
3,2.0,48.0,15.0,63.0,52.0,5.0,0.000000
4,2.0,48.0,15.0,63.0,56.0,5.0,0.000000


# Predict over large dataset

## Previously trained model

The [`map_partitions` method](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.Series.map_partitions) allows execution of arbitrary functions on the partitions of the Dask DataFrame. Remember these partitions are just pandas DataFrames, so any code that works with pandas works here! This enables us to execute a function that performs predictions with a pre-trained model.

First lets get a handle on how to use the `map_partitions` function with a toy example.

Grab one partition from the Dask DataFrame for testing

In [54]:
taxi_feat_part = taxi_feat.partitions[0].compute()
print(type(taxi_feat_part))
print(taxi_feat_part.shape)

<class 'pandas.core.frame.DataFrame'>
(717801, 7)


In [71]:
def myfunc(df):
    return df['pickup_weekday'] * 5

In [75]:
myfunc(taxi_feat_part)

0          5.0
1          5.0
2         20.0
3         10.0
4         10.0
          ... 
718872    20.0
718873    20.0
718874    20.0
718875    20.0
718876    20.0
Name: pickup_weekday, Length: 717801, dtype: float64

In [84]:
out = taxi_feat.map_partitions(myfunc)

In [85]:
out

Dask Series Structure:
npartitions=127
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: myfunc, 3810 tasks

In [86]:
out.head()

0     5.0
1     5.0
2    20.0
3    10.0
4    10.0
dtype: float64

Dask will attempt to infer the data type of the function used with `map_partitions`. To be more explict, you should pass a `meta=` argument describing the data type of the output.

In [87]:
out = taxi_feat.map_partitions(
    myfunc,
    meta=pd.Series(dtype='float64')
)

Now let's use `map_partitions` to make predictions from a previously trained model. We'll load the model that was trained with scikit-learn and saved in [02-single-node.ipynb](02-single-node.ipynb).

In [38]:
import cloudpickle

model = cloudpickle.load(open('/tmp/model.pkl', 'rb'))

### Exercise

Write a function that uses the `model` to make a prediction for a given input DataFrame, then execute it with `map_partitions` across the entire `taxi_feat` DataFrame. 

Assume the input DataFrame already has had features created. The output of the function should be a `pd.Series` object that has predictions for each row in the input DataFrame. Validate that your function works properly by executing it with `taxi_feat_part` as input before trying it with `map_partitions`. The output should look something like:

```
0         0.164296
1         0.166451
            ...   
717799    0.165269
717800    0.168916
Length: 717801, dtype: float64
```

In [ ]:
def predict(df):
    <FILL IN>
    
preds_sklearn = predict(taxi_feat_part)
preds_sklearn.head()

In [ ]:
preds_dask = taxi_feat.map_partitions(
    <FILL IN>
)
preds_dask.head()

In [92]:
def predict(df):
    preds = model.predict(df[features])
    return pd.Series(preds)

preds_sklearn = predict(taxi_feat_part)
preds_sklearn.head()

In [ ]:
preds_dask = taxi_feat.map_partitions(
    predict, 
    meta=pd.Series(dtype='float64'),
)
preds_dask.head()

In [94]:
len(preds_sklearn)

717801

In [95]:
len(preds_dask)

84194625

In [97]:
from dask_ml.metrics import mean_squared_error

mean_squared_error(
    taxi_feat[label].values, 
    preds_dask.values, 
    squared=False,
)

13.453046384464571

## `ParallelPostFit` wrapper

Dask ML also has a [`ParallelPostFit` meta-estimator](https://ml.dask.org/meta-estimators.html) the wraps a scikit-learn model for parallelized predictions. This is useful in scenarios where it is known up-front that a model needs to be trained on a small amount of data but predictions need to be made for a large amount of data.

In [99]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

from dask_ml.wrappers import ParallelPostFit

pipeline = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('clf', ElasticNet(normalize=False, max_iter=100, l1_ratio=0)),
])

ppf = ParallelPostFit(estimator=pipeline)
ppf_fitted = ppf.fit(taxi_feat_part[features], taxi_feat_part[label])

In [104]:
preds_dask = ppf_fitted.predict(taxi_feat[features])

mean_squared_error(
    taxi_feat[label].values,
    preds_dask, 
    squared=False,
)

13.453205303896075

## Train model with large dataset

First, we need to split our `taxi_feat` DataFrame into train/test sets.

### Exercise

Use the [`dask_ml.model_selection.train_test_split` function](https://ml.dask.org/modules/generated/dask_ml.model_selection.train_test_split.html) to split into train and test sets. (Hint: the `dask_ml` function works the same as the `sklearn` function.)

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = <FILL IN>

In [15]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    taxi_feat[features], 
    taxi_feat[label], 
    test_size=0.3,
    random_state=42
)

In [16]:
%%time
X_train, X_test, y_train, y_test = dask.persist(
    X_train, X_test, y_train, y_test,
)
_ = wait(X_train)

CPU times: user 218 ms, sys: 31.7 ms, total: 250 ms
Wall time: 5.89 s


In [17]:
len(X_train), len(y_train)

(58939024, 58939024)

In [18]:
len(X_test), len(y_test)

(25255601, 25255601)

## Dask ML models

The dask-ml package has parallel implementations of machine learning algorithms that do not have parallel implementations in scikit-learn or other packages. These currently cover linear models and clustering.

In [105]:
from sklearn.pipeline import Pipeline

from dask_ml.linear_model import LinearRegression
from dask_ml.preprocessing import StandardScaler
from dask_ml.metrics import mean_squared_error
from dask_ml.model_selection import GridSearchCV

lr = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('clf', LinearRegression(penalty='l2', max_iter=100)),
])

In [20]:
X_train_arr = X_train.to_dask_array(lengths=True)
y_train_arr = y_train.to_dask_array(lengths=True)
X_test_arr = X_test.to_dask_array(lengths=True)
y_test_arr = y_test.to_dask_array(lengths=True)

In [21]:
%%time

lr_fitted = lr.fit(
    X_train_arr,
    y_train_arr,
)

CPU times: user 31.1 s, sys: 1.04 s, total: 32.2 s
Wall time: 7min 36s


In [28]:
lr_preds = lr_fitted.predict(X_test_arr)
mean_squared_error(y_test_arr, lr_preds, squared=False)

15.544812999425133

## XGBoost

The `dask-xgboost` package has an integration between XGBoost and Dask that parallelizes model training and prediction across a Dask cluster. 

> Note: The native XGBoost library also has an integration in the `xgboost.dask` module that will become the recommended approach in the future.

In [22]:
from dask_xgboost import XGBRegressor

xgb = XGBRegressor(
    objective="reg:squarederror",
    tree_method='approx',
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
)

In [23]:
%%time

xgb_fitted = xgb.fit(
    X_train_arr,
    y_train_arr,
)

CPU times: user 579 ms, sys: 69.4 ms, total: 649 ms
Wall time: 14min 55s


In [24]:
xgb_preds = xgb_fitted.predict(X_test_arr)
mean_squared_error(y_test_arr, xgb_preds, squared=False)

15.536646029575314

### Incremental learning

Dask ML can hook into scikit-learn's incremental training features with the [`Incremental` meta-estimator](https://ml.dask.org/incremental.html). Any model that implements a `partial_fit()` method can be utilized with this meta-estimator. We will not cover `Incremental` in this tutorial (`ElasticNet` does not have a `partial_fit()` method). 

### Scikit-learn models + Joblib

For scikit-learn models that have parallel implementations, a joblib backend for Dask can be utilized to train on large datasets. This follows the same process as shown in [03-hyperparameter.ipynb](03-hyperparameter.ipynb), except the work must be done inside a [future](https://docs.dask.org/en/latest/futures.html) to bypass sending all the data through the scheduler. This is outside of the scope of this workshop, but it would look something like:

```python
from sklearn.ensemble import RandomForestRegressor
from dask.distributed import worker_client
import joblib

def train_rf():
    with worker_client() as client:
        X_train, y_train = ...

        rf = RandomForestRegressor(max_depth=3, n_estimators=100)
        with joblib.parallel_backend('dask', client=client):
            rf.fit(X_train, y_train)
            
        return rf
    
fut = client.submit(train_rf)
fut.result()  # block until function is done
```